# Data Curation Final Project

## Setting the database

Make sure you have mysql available

In [8]:
!mysql --version

mysql  Ver 8.0.33 for Win64 on x86_64 (MySQL Community Server - GPL)


Creating database and schema.

_(don't use this database as an example of how I would handle a real password, please)_

In [ ]:
import mysql.connector


query = """
CREATE DATABASE IF NOT EXISTS south_tyrol_hazards;
USE south_tyrol_hazards;


CREATE TABLE IF NOT EXISTS Danger_levels (
    CODE INT UNSIGNED PRIMARY KEY,
    LABEL_IT VARCHAR(50),
    LABEL_DE VARCHAR(50)
);

DELIMITER $$

-- Optimized trigger for hazard zone inserts
CREATE TRIGGER after_hazard_zone_insert
AFTER INSERT ON Hazard_zones
FOR EACH ROW
BEGIN
    -- Municipality-constrained spatial intersection (MAJOR performance boost)
    INSERT INTO Road_Hazard_Zone_intersection (ID_HAZARD, ID_ROAD)
    SELECT NEW.FID, r.FID
    FROM Road_segments r 
    WHERE r.ISTAT_CODE = NEW.ISTAT_CODE                    -- Municipality filter FIRST (indexed)
    AND MBRIntersects(NEW.SHAPE, r.geom)                  -- Fast spatial bounding box check
    AND ST_Intersects(NEW.SHAPE, r.geom)                  -- Precise spatial intersection
    AND NOT EXISTS (
        SELECT 1 FROM Road_Hazard_Zone_intersection 
        WHERE ID_HAZARD = NEW.FID AND ID_ROAD = r.FID
    );
END$$

-- Optimized trigger for hazard zone updates
CREATE TRIGGER after_hazard_zone_update
AFTER UPDATE ON Hazard_zones
FOR EACH ROW
BEGIN
    -- Only process if geometry or municipality changed
    IF NOT ST_Equals(OLD.SHAPE, NEW.SHAPE) OR OLD.ISTAT_CODE != NEW.ISTAT_CODE THEN
        -- Delete old intersections (indexed on ID_HAZARD)
        DELETE FROM Road_Hazard_Zone_intersection 
        WHERE ID_HAZARD = NEW.FID;
        
        -- Add new intersections with municipality constraint
        INSERT INTO Road_Hazard_Zone_intersection (ID_HAZARD, ID_ROAD)
        SELECT NEW.FID, r.FID
        FROM Road_segments r
        WHERE r.ISTAT_CODE = NEW.ISTAT_CODE                -- Municipality filter reduces search space dramatically
        AND MBRIntersects(NEW.SHAPE, r.geom)              -- Spatial index pre-filter
        AND ST_Intersects(NEW.SHAPE, r.geom);             -- Precise intersection
    END IF;
END$$

-- Optimized trigger for road segment inserts
CREATE TRIGGER after_road_segment_insert
AFTER INSERT ON Road_segments
FOR EACH ROW
BEGIN
    -- Municipality-constrained intersection lookup
    INSERT INTO Road_Hazard_Zone_intersection (ID_HAZARD, ID_ROAD)
    SELECT h.FID, NEW.FID
    FROM Hazard_zones h
    WHERE h.ISTAT_CODE = NEW.ISTAT_CODE                    -- Same municipality only (indexed lookup)
    AND MBRIntersects(h.SHAPE, NEW.geom)                  -- Spatial bounding box pre-filter
    AND ST_Intersects(h.SHAPE, NEW.geom)                  -- Precise spatial intersection
    AND NOT EXISTS (
        SELECT 1 FROM Road_Hazard_Zone_intersection 
        WHERE ID_HAZARD = h.FID AND ID_ROAD = NEW.FID
    );
END$$

-- Optimized trigger for road segment updates
CREATE TRIGGER after_road_segment_update
AFTER UPDATE ON Road_segments
FOR EACH ROW
BEGIN
    -- Process if geometry or municipality assignment changed
    IF NOT ST_Equals(OLD.geom, NEW.geom) OR OLD.ISTAT_CODE != NEW.ISTAT_CODE THEN
        -- Delete old intersections (indexed on ID_ROAD)
        DELETE FROM Road_Hazard_Zone_intersection 
        WHERE ID_ROAD = NEW.FID;
        
        -- Add new intersections with municipality optimization
        INSERT INTO Road_Hazard_Zone_intersection (ID_HAZARD, ID_ROAD)
        SELECT h.FID, NEW.FID
        FROM Hazard_zones h
        WHERE h.ISTAT_CODE = NEW.ISTAT_CODE                -- Municipality constraint first
        AND MBRIntersects(h.SHAPE, NEW.geom)              -- Then spatial filtering
        AND ST_Intersects(h.SHAPE, NEW.geom);
    END IF;
END$$

DELIMITER ;
"""


def create_database():
    """Execute the SQL file to create the database"""
    
    # Connect to MySQL
    conn = mysql.connector.connect(
        host='localhost',
        user='root',
        password='1234'
    )
    
    cursor = conn.cursor()
    
    # Read and execute the SQL file
    with open('RDB files/create database.sql', 'r', encoding='utf-8') as file:
        sql_script = file.read()
    sql_script = query
    # Split into parts: before triggers, triggers, after triggers
    parts = sql_script.split('DELIMITER $$')
    
    # Execute statements before triggers (split by semicolon)
    before_triggers = parts[0]
    statements = [stmt.strip() for stmt in before_triggers.split(';') if stmt.strip()]
    
    for stmt in statements:
        if stmt:
            cursor.execute(stmt)
    
    # Execute triggers (split by $$)
    if len(parts) > 1:
        trigger_section = parts[1].split('DELIMITER ;')[0]
        triggers = [trigger.strip() for trigger in trigger_section.split('$$') if trigger.strip()]
        
        for trigger in triggers:
            if trigger and trigger.startswith('CREATE TRIGGER'):
                cursor.execute(trigger)
                
    print("Database created successfully!")
    
    # Verify
    cursor.execute("USE south_tyrol_hazards")
    
    cursor.execute("SHOW TABLES")
    tables = cursor.fetchall()
    print(f"Created {len(tables)} tables")
    
    conn.close()

# Run it
create_database()

Database created successfully!
Created 11 tables


Let's check if it worked

In [2]:
import mysql.connector

def show_database_schema():
    """Display the complete schema of the south_tyrol_hazards database"""
    
    conn = mysql.connector.connect(
        host='localhost',
        user='root',
        password='1234',
        database='south_tyrol_hazards'
    )
    
    cursor = conn.cursor()
    
    print("=" * 60)
    print("SOUTH TYROL HAZARDS DATABASE SCHEMA")
    print("=" * 60)
    
    # Get all tables
    cursor.execute("SHOW TABLES")
    tables = cursor.fetchall()
    
    print(f"\nDatabase contains {len(tables)} tables:\n")
    
    for (table_name,) in tables:
        print(f"📋 TABLE: {table_name}")
        print("-" * 50)
        
        # Get table structure
        cursor.execute(f"DESCRIBE {table_name}")
        columns = cursor.fetchall()
        
        for column in columns:
            field, type_, null, key, default, extra = column
            key_info = f" [{key}]" if key else ""
            null_info = " NULL" if null == "YES" else " NOT NULL"
            default_info = f" DEFAULT {default}" if default else ""
            extra_info = f" {extra}" if extra else ""
            
            print(f"  {field}: {type_}{key_info}{null_info}{default_info}{extra_info}")
        
        print()
    
    # Show foreign key relationships
    print("\n🔗 FOREIGN KEY RELATIONSHIPS:")
    print("-" * 50)
    
    cursor.execute("""
        SELECT 
            TABLE_NAME,
            COLUMN_NAME,
            REFERENCED_TABLE_NAME,
            REFERENCED_COLUMN_NAME
        FROM INFORMATION_SCHEMA.KEY_COLUMN_USAGE
        WHERE REFERENCED_TABLE_SCHEMA = 'south_tyrol_hazards'
        AND REFERENCED_TABLE_NAME IS NOT NULL
    """)
    
    foreign_keys = cursor.fetchall()
    
    for fk in foreign_keys:
        table, column, ref_table, ref_column = fk
        print(f"  {table}.{column} → {ref_table}.{ref_column}")
    
    # Show triggers
    print(f"\n⚡ TRIGGERS:")
    print("-" * 50)
    
    cursor.execute("SHOW TRIGGERS")
    triggers = cursor.fetchall()
    
    for trigger in triggers:
        trigger_name = trigger[0]
        event = trigger[1]  # INSERT, UPDATE, DELETE
        table = trigger[2]
        timing = trigger[4]  # BEFORE, AFTER
        print(f"  {trigger_name}: {timing} {event} on {table}")
    
    # Show indexes
    print(f"\n📊 INDEXES:")
    print("-" * 50)
    
    for (table_name,) in tables:
        cursor.execute(f"SHOW INDEX FROM {table_name}")
        indexes = cursor.fetchall()
        
        table_indexes = {}
        for idx in indexes:
            index_name = idx[2]
            column_name = idx[4]
            index_type = idx[10] if len(idx) > 10 else "BTREE"
            
            if index_name not in table_indexes:
                table_indexes[index_name] = []
            table_indexes[index_name].append(column_name)
        
        if table_indexes:
            print(f"  {table_name}:")
            for index_name, columns in table_indexes.items():
                cols_str = ", ".join(columns)
                print(f"    {index_name}: ({cols_str})")
    
    conn.close()
    print("\n" + "=" * 60)

# Show the schema
show_database_schema()

SOUTH TYROL HAZARDS DATABASE SCHEMA

Database contains 10 tables:

📋 TABLE: danger_levels
--------------------------------------------------
  CODE: int unsigned [PRI] NOT NULL
  LABEL_IT: varchar(50) NULL
  LABEL_DE: varchar(50) NULL

📋 TABLE: districts
--------------------------------------------------
  CODE: tinyint unsigned [PRI] NOT NULL
  LABEL_IT: varchar(100) NULL
  LABEL_DE: varchar(100) NULL

📋 TABLE: hazard_zones
--------------------------------------------------
  FID: varchar(50) [PRI] NOT NULL
  OBJECTID: int unsigned [UNI] NULL
  ISTAT_CODE: smallint unsigned [MUL] NULL
  CODE_PROCESS: varchar(2) [MUL] NULL
  CODE_STUDY: varchar(1) [MUL] NULL
  CODE_DANGER: int unsigned [MUL] NULL
  X_LABEL: decimal(9,2) NULL
  Y_LABEL: decimal(9,2) NULL
  SHAPE: geometry [MUL] NOT NULL

📋 TABLE: health_districts
--------------------------------------------------
  HEALTH_DISTRICT: tinyint unsigned [PRI] NOT NULL
  HEALTH_REGION: tinyint unsigned NULL

📋 TABLE: municipalities
----------

Beautiful.